In [1]:
import sys
import subprocess

print(f"Python: {sys.executable}")

# Uninstall NumPy 2.x
print("Uninstalling NumPy 2.x...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"], check=True)

# Install NumPy 1.26.4
print("Installing NumPy 1.26.4...")
subprocess.run([sys.executable, "-m", "pip", "install", "numpy==1.26.4"], check=True)

print("\n✅ DONE! Now restart kernel: Kernel → Restart Kernel")

Python: /Applications/Xcode.app/Contents/Developer/usr/bin/python3
Uninstalling NumPy 2.x...
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Installing NumPy 1.26.4...
Defaulting to user installation because normal site-packages is not writeable


  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)



✅ DONE! Now restart kernel: Kernel → Restart Kernel


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 1.1.0 requires tenacity>=8.2.3, but you have tenacity 8.2.0 which is incompatible.
great-expectations 0.17.0 requires pydantic<2.0,>=1.9.2, but you have pydantic 2.12.3 which is incompatible.
opencv-python-headless 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [ ]:
import sys
import subprocess

# Uninstall broken packages
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "scipy", "albumentations", "opencv-python", "opencv-python-headless"])

# Install in correct order
subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "scipy>=1.7.0,<1.15.0"])
subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "opencv-python-headless>=4.5.0"])
subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "albumentations>=1.3.0"])

print("✅ Done! Now restart the kernel: Kernel → Restart Kernel")

Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1
Found existing installation: albumentations 2.0.8
Uninstalling albumentations-2.0.8:
  Successfully uninstalled albumentations-2.0.8
Found existing installation: opencv-python-headless 4.13.0.92
Uninstalling opencv-python-headless-4.13.0.92:
  Successfully uninstalled opencv-python-headless-4.13.0.92


Defaulting to user installation because normal site-packages is not writeable


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 45.1 MB/s  0:00:00 eta 0:00:01


In [ ]:
import subprocess
import sys
import os

print("Installing dependencies for Sustainability AI Model (MacBook Local Training)...")
print(f"Python version: {sys.version}")
print("="*60)

# Install packages one by one with error handling
def install_package(package_spec, description=""):
    """Install a package with error handling."""
    try:
        print(f"Installing {description or package_spec}...")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", "--upgrade"] + package_spec.split(),
            timeout=300  # 5 minute timeout per package
        )
        print(f"  ✅ {description or package_spec}")
        return True
    except subprocess.TimeoutExpired:
        print(f"  ⚠️  Timeout installing {description or package_spec}, skipping...")
        return False
    except subprocess.CalledProcessError as e:
        print(f"  ⚠️  Failed to install {description or package_spec}: {e}")
        return False

# Upgrade pip first
print("Upgrading pip...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pip"], timeout=60)

# Install Kaggle API
install_package("kaggle", "Kaggle API")

# Install core dependencies (Python 3.9 compatible versions)
install_package("numpy>=1.19.0,<2.0", "NumPy")
install_package("scipy>=1.7.0,<1.15.0", "SciPy")
install_package("Pillow>=8.0.0", "Pillow")
install_package("pandas>=1.3.0", "Pandas")
install_package("scikit-learn>=1.0.0", "scikit-learn")
install_package("matplotlib>=3.4.0", "Matplotlib")
install_package("seaborn>=0.11.0", "Seaborn")
install_package("tqdm>=4.62.0", "tqdm")

# Install PyTorch with compatible torchvision version
print("Checking PyTorch installation...")
try:
    import torch
    import torchvision
    torch_version = torch.__version__
    torchvision_version = torchvision.__version__
    print(f"  Current PyTorch: {torch_version}")
    print(f"  Current torchvision: {torchvision_version}")

    # Check if versions are compatible
    # PyTorch 2.x needs torchvision 0.15+
    # PyTorch 1.x needs torchvision 0.x
    torch_major = int(torch_version.split('.')[0])
    tv_major = int(torchvision_version.split('.')[0])

    if torch_major == 2 and tv_major == 0 and int(torchvision_version.split('.')[1]) < 15:
        print("  ⚠️  Version mismatch detected! Reinstalling compatible versions...")
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision"], check=False)
        install_package("torch==2.0.1 torchvision==0.15.2", "PyTorch 2.0.1 + torchvision 0.15.2")
    elif torch_major != tv_major:
        print("  ⚠️  Major version mismatch! Reinstalling compatible versions...")
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision"], check=False)
        install_package("torch==2.0.1 torchvision==0.15.2", "PyTorch 2.0.1 + torchvision 0.15.2")
    else:
        print("  ✅ PyTorch and torchvision versions are compatible")

except ImportError:
    print("  Installing PyTorch and torchvision...")
    install_package("torch==2.0.1 torchvision==0.15.2", "PyTorch 2.0.1 + torchvision 0.15.2")
except AttributeError as e:
    print(f"  ⚠️  Version compatibility issue detected: {e}")
    print("  Reinstalling compatible PyTorch and torchvision versions...")
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision"], check=False)
    install_package("torch==2.0.1 torchvision==0.15.2", "PyTorch 2.0.1 + torchvision 0.15.2")

# Install timm (Python 3.9 compatible)
install_package("timm>=0.9.0", "timm")

# Install albumentations
install_package("albumentations>=1.3.0", "Albumentations")

# Install other dependencies
install_package("einops>=0.6.0", "einops")
install_package("wandb>=0.15.0", "Weights & Biases")

# Install PyTorch Geometric (simplified for Python 3.9)
print("Installing PyTorch Geometric...")
install_package("torch-geometric", "PyTorch Geometric")

# Try to install torch-scatter and torch-sparse (optional, may fail on some systems)
print("Installing optional PyG dependencies (may fail, that's OK)...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", "torch-scatter", "torch-sparse"],
        timeout=300,
        check=False  # Don't fail if this doesn't work
    )
    print("  ✅ torch-scatter and torch-sparse installed")
except:
    print("  ⚠️  torch-scatter/torch-sparse installation skipped (optional)")

print("="*60)
print("✅ Core dependencies installed successfully!")
print("="*60)


In [ ]:
# KAGGLE API SETUP AND DATASET DOWNLOAD
import os
import json
from pathlib import Path
import zipfile
import shutil

print("="*80)
print("🔑 CONFIGURING KAGGLE API")
print("="*80)
print()


KAGGLE_USERNAME = "michealjiang"  # Your Kaggle username
KAGGLE_KEY = "92ce58a4cc3d98ed20dca81b8598123f"  # Your Kaggle API key

# Alternative: If you already have kaggle.json, we can read it
kaggle_json_path = Path.home() / ".kaggle" / "kaggle.json"
if kaggle_json_path.exists():
    print("📄 Found existing kaggle.json, loading credentials...")
    with open(kaggle_json_path, 'r') as f:
        existing_creds = json.load(f)
        KAGGLE_USERNAME = existing_creds.get("username", KAGGLE_USERNAME)
        KAGGLE_KEY = existing_creds.get("key", KAGGLE_KEY)
    print(f"   ✅ Loaded username: {KAGGLE_USERNAME}")
else:
    print("📝 No existing kaggle.json found, using credentials from above...")

# Validate credentials
if KAGGLE_USERNAME == "YOUR_KAGGLE_USERNAME" or KAGGLE_KEY == "YOUR_KAGGLE_API_KEY":
    print()
    print("="*80)
    print("⚠️  ERROR: KAGGLE CREDENTIALS NOT SET!")
    print("="*80)
    print()
    print("Please follow these steps:")
    print()
    print("1. Go to: https://www.kaggle.com/settings")
    print("2. Scroll to 'API' section")
    print("3. Click 'Create New Token'")
    print("4. This downloads 'kaggle.json' to your Downloads folder")
    print("5. Open kaggle.json and you'll see:")
    print('   {"username":"your_username","key":"your_api_key"}')
    print()
    print("6. Copy those values and paste them in the cell above:")
    print('   KAGGLE_USERNAME = "your_username"')
    print('   KAGGLE_KEY = "your_api_key"')
    print()
    print("7. Re-run this cell")
    print()
    print("="*80)
    raise ValueError("Kaggle credentials not configured. Please set KAGGLE_USERNAME and KAGGLE_KEY above.")

print()

# Create ~/.kaggle directory if it doesn't exist
kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

# Create kaggle.json with credentials
kaggle_json_path = kaggle_dir / "kaggle.json"
kaggle_credentials = {
    "username": KAGGLE_USERNAME,
    "key": KAGGLE_KEY
}

# Write credentials to file
with open(kaggle_json_path, 'w') as f:
    json.dump(kaggle_credentials, f, indent=2)

# Set proper permissions (required by Kaggle API on Unix systems)
try:
    os.chmod(kaggle_json_path, 0o600)
except:
    pass  # Windows doesn't support chmod

print(f"✅ Kaggle credentials saved to: {kaggle_json_path}")
print(f"   Username: {KAGGLE_USERNAME}")
print(f"   Key: {KAGGLE_KEY[:10]}...{KAGGLE_KEY[-4:]}")
print()
print("="*80)
print()

# Kaggle datasets to download
KAGGLE_DATASETS = [
    {"slug": "sumn2u/garbage-classification-v2", "name": "garbage-classification-v2"},
    {"slug": "zlatan599/garbage-dataset-classification", "name": "garbage-dataset-classification"},
    {"slug": "parohod/warp-waste-recycling-plant-dataset", "name": "warp-waste-recycling-plant-dataset"},
    {"slug": "asdasdasasdas/garbage-classification", "name": "garbage-classification"},
    {"slug": "techsash/waste-classification-data", "name": "waste-classification-data"},
    {"slug": "alistairking/recyclable-and-household-waste-classification", "name": "recyclable-and-household-waste-classification"},
    {"slug": "vishallazrus/multi-class-garbage-classification-dataset", "name": "multi-class-garbage-classification-dataset"},
    {"slug": "mostafaabla/garbage-classification", "name": "garbage-classification-mostafa"}
]

def download_kaggle_datasets(datasets, base_dir="./data/kaggle"):
    """
    Download Kaggle datasets using the Kaggle Python API (not CLI).

    Args:
        datasets: List of dataset dictionaries with 'slug' and 'name'
        base_dir: Base directory to store downloaded datasets
    """
    import sys
    import time

    base_path = Path(base_dir)
    base_path.mkdir(parents=True, exist_ok=True)

    print("="*80)
    print("📦 KAGGLE DATASET DOWNLOAD")
    print("="*80)

    # Import Kaggle API
    try:
        from kaggle.api.kaggle_api_extended import KaggleApi
        api = KaggleApi()
        api.authenticate()
        print("✅ Kaggle API authenticated successfully!")
    except Exception as e:
        print(f"❌ Failed to authenticate Kaggle API: {e}")
        print("\nPlease ensure:")
        print("1. You have a Kaggle account")
        print("2. Your username is correct in the cell above")
        print("3. Your API key is correct")
        return [], datasets

    print()

    downloaded = []
    failed = []

    for idx, dataset in enumerate(datasets, 1):
        dataset_slug = dataset["slug"]
        dataset_name = dataset["name"]
        dataset_path = base_path / dataset_name

        print(f"\n[{idx}/{len(datasets)}] {dataset_name}")
        print(f"      Source: {dataset_slug}")

        # Check if already downloaded
        if dataset_path.exists() and any(dataset_path.iterdir()):
            print(f"      ✅ Already downloaded, skipping...")
            downloaded.append(dataset_name)
            continue

        print(f"      📥 Downloading...", end="", flush=True)

        try:
            # Create dataset directory
            dataset_path.mkdir(parents=True, exist_ok=True)

            start_time = time.time()

            # Download using Kaggle Python API with quiet mode
            # This prevents blocking output
            api.dataset_download_files(
                dataset_slug,
                path=str(dataset_path),
                unzip=True,
                quiet=True  # Changed to True to prevent blocking
            )

            elapsed = time.time() - start_time

            # Verify download
            if dataset_path.exists() and any(dataset_path.iterdir()):
                print(f" ✅ Done! ({elapsed:.1f}s)")
                downloaded.append(dataset_name)
            else:
                print(f" ❌ Failed (no files found)")
                failed.append(dataset_name)

        except KeyboardInterrupt:
            print(f"\n\n⚠️  Download interrupted by user!")
            print(f"   Downloaded so far: {len(downloaded)}/{len(datasets)}")
            return downloaded, failed + [d["name"] for d in datasets[idx-1:]]

        except Exception as e:
            error_msg = str(e)
            # Shorten long error messages
            if len(error_msg) > 100:
                error_msg = error_msg[:100] + "..."
            print(f" ❌ Error: {error_msg}")
            failed.append(dataset_name)

            # Clean up partial download
            if dataset_path.exists():
                try:
                    shutil.rmtree(dataset_path)
                except:
                    pass

        # Flush output to ensure it's displayed
        sys.stdout.flush()

    print("\n" + "="*80)
    print("📊 DOWNLOAD SUMMARY")
    print("="*80)
    print(f"✅ Successfully downloaded: {len(downloaded)}/{len(datasets)}")
    print(f"❌ Failed: {len(failed)}/{len(datasets)}")

    if downloaded:
        print(f"\n✅ Downloaded datasets:")
        for name in downloaded:
            print(f"   ✓ {name}")

    if failed:
        print(f"\n⚠️  Failed datasets:")
        for name in failed:
            print(f"   ✗ {name}")

    print("="*80)

    return downloaded, failed

# Download all datasets
print("Starting Kaggle dataset downloads...")
print("This may take 10-30 minutes depending on your internet connection.")
print("💡 TIP: If a download seems stuck, press Ctrl+C to skip and continue with next dataset")
print()

try:
    downloaded, failed = download_kaggle_datasets(KAGGLE_DATASETS)
except KeyboardInterrupt:
    print("\n\n⚠️  Download process interrupted!")
    print("You can continue with the datasets that were successfully downloaded.")
    downloaded, failed = [], []

if len(downloaded) == 0:
    print("\n⚠️  WARNING: No datasets were downloaded!")
    print("Please check your Kaggle API token and internet connection.")
    print("\n💡 You can still continue with the notebook - we'll use sample data for testing.")
else:
    print(f"\n✅ Ready to proceed with {len(downloaded)} datasets!")

print("\n" + "="*80)
print("📝 NOTE: Cell execution complete! You can now proceed to the next cell.")
print("="*80)



In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import json
import random
import logging
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

import timm
from timm.data import create_transform, resolve_data_config
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from tqdm.notebook import tqdm
import wandb
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def set_seed(seed: int = 42):
    """Set random seed for reproducibility across all frameworks."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        # MPS doesn't need special seeding
        pass
    logger.info(f"✓ Random seed set to {seed}")

def get_device():
    """
    Detect and return the best available device for training.
    Priority: CUDA > MPS (Apple Silicon) > CPU

    CRITICAL: MPS DISABLED due to crashes with NumPy 2.x + manual tensor conversion
    Using CPU for 100% stability until NumPy is fixed in Jupyter environment
    """
    if torch.cuda.is_available():
        device = torch.device("cuda")
        logger.info(f"🚀 Using CUDA GPU: {torch.cuda.get_device_name(0)}")
        logger.info(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        return device
    # CRITICAL FIX: MPS disabled - causes Python crashes with NumPy 2.x
    # elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    #     device = torch.device("mps")
    #     logger.info(f"🍎 Using Apple Silicon MPS (Metal Performance Shaders)")
    #     logger.info(f"   Optimized for M1/M2/M3 chips")
    #     return device
    else:
        device = torch.device("cpu")
        logger.info(f"💻 Using CPU for maximum stability")
        logger.info(f"   ⚠️  MPS disabled due to crashes - will re-enable after NumPy fix")
        logger.info(f"   Training will be slower but 100% stable")
        return device

def optimize_memory(device):
    """
    Memory optimization for different hardware backends.
    Supports CUDA, MPS (Apple Silicon), and CPU.
    """
    if device.type == "cuda":
        # CUDA GPU optimization
        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = True

        # TF32 support (only in PyTorch 1.7+)
        try:
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
        except AttributeError:
            pass  # Older PyTorch versions don't have TF32 support

        import os
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,max_split_size_mb:512'

        try:
            torch.cuda.set_per_process_memory_fraction(0.95)
        except AttributeError:
            pass  # Older PyTorch versions don't have this method

        logger.info("✓ CUDA memory optimization enabled")

    elif device.type == "mps":
        # MPS (Apple Silicon) optimization
        # MPS doesn't have explicit memory management like CUDA
        # But we can set environment variables for better performance
        import os
        os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'  # Disable memory caching

        logger.info("✓ MPS optimization enabled")
        logger.info("  - High watermark ratio: 0.0 (aggressive memory release)")

    else:
        # CPU optimization
        # Set number of threads for CPU training
        import os
        num_threads = os.cpu_count() or 4
        torch.set_num_threads(num_threads)

        logger.info(f"✓ CPU optimization enabled")
        logger.info(f"  - Using {num_threads} threads")

class EarlyStopping:
    def __init__(self, patience=15, mode="max", delta=0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.mode = mode
        self.delta = delta

    def __call__(self, current_score):
        if self.best_score is None:
            self.best_score = current_score
        elif self.mode == "max":
            if current_score <= self.best_score + self.delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = current_score
                self.counter = 0
        return self.early_stop

In [ ]:
TARGET_CLASSES = [
    'aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging',
    'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'egg_shells', 'food_waste',
    'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines',
    'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles',
    'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws',
    'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups',
    'styrofoam_food_containers', 'tea_bags'
]

VISION_CONFIG = {
    "model": {
        "backbone": "eva02_large_patch14_224",  # EVA02 Large (304M params) - PRODUCTION GRADE
        "pretrained": True,  # CRITICAL: Must use pretrained weights
        "num_classes": 30,
        "drop_rate": 0.0,  # ULTRA-OPTIMIZED: NO dropout for pretrained model (already robust)
        "drop_path_rate": 0.0  # ULTRA-OPTIMIZED: NO drop_path (pretrained model doesn't need it)
    },
    "data": {
        "input_size": 224,  # Standard EVA02 input size
        "num_workers": 0,  # CRITICAL: Disabled for macOS stability
        "pin_memory": False,  # Not needed for CPU
        "sources": [
            {
                "name": "master_30",
                "path": "./data/kaggle/recyclable-and-household-waste-classification/images",
                "type": "master"
            },
            {
                "name": "garbage_12",
                "path": "./data/kaggle/garbage-classification-mostafa/garbage_classification",
                "type": "mapped_12"
            },
            {
                "name": "waste_22k",
                "path": "./data/kaggle/waste-classification-data/DATASET",
                "type": "mapped_2"
            },
            {
                "name": "garbage_v2_10",
                "path": "./data/kaggle/garbage-classification-v2",
                "type": "mapped_10"
            },
            {
                "name": "garbage_6",
                "path": "./data/kaggle/garbage-classification",
                "type": "mapped_6"
            },
            {
                "name": "garbage_balanced",
                "path": "./data/kaggle/garbage-dataset-classification",
                "type": "mapped_6"
            },
            {
                "name": "warp_industrial",
                "path": "./data/kaggle/warp-waste-recycling-plant-dataset",
                "type": "industrial"
            },
            {
                "name": "multiclass_garbage",
                "path": "./data/kaggle/multi-class-garbage-classification-dataset",
                "type": "multiclass"
            }
        ]
    },
    "training": {
        "batch_size": 32,  # ULTRA-OPTIMIZED: Larger batch for stable gradients (2x increase)
        "grad_accum_steps": 2,  # ULTRA-OPTIMIZED: Maintain effective batch size of 64 (32 × 2)
        "learning_rate": 1e-3,  # ULTRA-OPTIMIZED: 2x higher for faster convergence (was 5e-4)
        "weight_decay": 0.0,  # ULTRA-OPTIMIZED: NO weight decay (pretrained model doesn't need it)
        "num_epochs": 20,
        "patience": 10,  # INCREASED: More patience for best convergence
        "use_amp": False,  # AMP not supported on CPU
        "max_grad_norm": 5.0,  # INCREASED: Allow larger gradients for faster learning
        "warmup_epochs": 0.5  # ULTRA-OPTIMIZED: Very fast warmup (half epoch)
    }
}

In [ ]:
class UnifiedWasteDataset(Dataset):
    """
    A unified dataset that ingests data from multiple sources and maps them
    to a single 30-class target schema.
    """
    def __init__(self, sources_config, target_classes, transform=None):
        self.transform = transform
        self.target_classes = sorted(target_classes)
        self.class_to_idx = {c: i for i, c in enumerate(self.target_classes)}
        self.samples = []

        self.skipped_count = 0
        self.skipped_labels = {}  # Track what labels are being skipped

        total_added = 0
        total_skipped = 0

        for source in sources_config:
            added, skipped = self._ingest_source(source)
            total_added += added
            total_skipped += skipped

        logger.info(f"="*60)
        logger.info(f"📊 Dataset Summary:")
        logger.info(f"  ✓ Total images loaded: {len(self.samples)}")
        logger.info(f"  ✓ Images added: {total_added}")
        logger.info(f"  ⚠ Images skipped: {total_skipped}")
        logger.info(f"  📈 Utilization: {100*total_added/(total_added+total_skipped) if (total_added+total_skipped) > 0 else 0:.1f}%")
        logger.info(f"="*60)

        # Log skipped labels for debugging (top 10 only)
        if self.skipped_labels:
            logger.warning(f"⚠ Top 10 skipped labels:")
            for label, count in sorted(self.skipped_labels.items(), key=lambda x: x[1], reverse=True)[:10]:
                logger.warning(f"  '{label}': {count} images")

        # Validate we have enough data
        if len(self.samples) == 0:
            raise ValueError(
                "❌ No images loaded! Please check:\n"
                "  1. Dataset paths are correct\n"
                "  2. Datasets are attached in Kaggle\n"
                "  3. Label mappings are configured correctly"
            )

        if len(self.samples) < 100:
            logger.warning(f"⚠ Very few images loaded ({len(self.samples)}). Training may not be effective.")

    def _ingest_source(self, source):
        """
        Ingest images from a data source with robust error handling.
        Returns: (images_added, images_skipped) tuple
        """
        path = Path(source["path"])
        images_added = 0
        images_skipped = 0

        if not path.exists():
            parent = path.parent
            found = False
            if parent.exists():
                for child in parent.iterdir():
                    if child.is_dir():
                        try:
                            if any(child.iterdir()):
                                path = child
                                found = True
                                break
                        except PermissionError:
                            continue

            if not found or not path.exists():
                logger.warning(f"⚠ Source {source['name']} not found at {source['path']}. Skipping.")
                return images_added, images_skipped

        logger.info(f"📂 Ingesting {source['name']} from {path}...")

        for root, _, files in os.walk(path):
            folder_name = Path(root).name.lower()

            target_label = self._map_label(folder_name, source['type'])

            if target_label:
                target_idx = self.class_to_idx[target_label]
                for file in files:
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                        self.samples.append((Path(root) / file, target_idx))
                        images_added += 1
            else:
                img_count = sum(1 for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')))
                if img_count > 0:
                    self.skipped_count += img_count
                    images_skipped += img_count
                    # Track which labels are being skipped
                    if folder_name not in self.skipped_labels:
                        self.skipped_labels[folder_name] = 0
                    self.skipped_labels[folder_name] += img_count

        logger.info(f"✓ {source['name']}: Added {images_added} images, skipped {images_skipped}")
        return images_added, images_skipped

    def _map_label(self, raw_label, source_type):
        """
        Professional label mapping with comprehensive coverage.
        Maps diverse dataset labels to unified 30-class taxonomy.
        """
        raw = raw_label.lower().strip()

        # Skip metadata/structure folders that are not actual labels
        metadata_folders = {
            'default', 'real_world', 'images', 'train', 'test', 'val',
            'segmentationobject', 'segmentationclass', 'jpegimages',
            'annotations', 'assets', 'data', 'dataset', 'samples'
        }
        if raw in metadata_folders:
            return None

        if source_type == 'master':
            if raw in self.target_classes:
                return raw
            # Fallback: try to find closest match
            for target in self.target_classes:
                if raw in target or target in raw:
                    return target
            return None

        if source_type == 'mapped_12':
            mapping = {
                'paper': 'office_paper',
                'cardboard': 'cardboard_boxes',
                'plastic': 'plastic_food_containers',
                'metal': 'aluminum_food_cans',
                'glass': 'glass_food_jars',
                'brown-glass': 'glass_beverage_bottles',
                'green-glass': 'glass_beverage_bottles',
                'white-glass': 'glass_food_jars',
                'clothes': 'clothing',
                'shoes': 'shoes',
                'biological': 'food_waste',
                'trash': 'food_waste'
            }
            return mapping.get(raw)

        if source_type == 'mapped_2':
            # Organic waste
            if raw in ['organic', 'o']:
                return 'food_waste'
            # Recyclable waste (paper, plastic, metal, glass mix)
            if raw in ['recyclable', 'r']:
                return 'plastic_food_containers'  # Generic recyclable
            return None

        if source_type == 'mapped_10':
            mapping = {
                'metal': 'aluminum_food_cans',
                'glass': 'glass_food_jars',
                'biological': 'food_waste',
                'paper': 'office_paper',
                'battery': 'aerosol_cans',
                'trash': 'food_waste',
                'cardboard': 'cardboard_boxes',
                'shoes': 'shoes',
                'clothes': 'clothing',
                'plastic': 'plastic_food_containers'
            }
            return mapping.get(raw)

        if source_type == 'mapped_6':
            mapping = {
                'cardboard': 'cardboard_boxes',
                'glass': 'glass_food_jars',
                'metal': 'aluminum_food_cans',
                'paper': 'office_paper',
                'plastic': 'plastic_food_containers',
                'trash': 'food_waste'
            }
            return mapping.get(raw)

        if source_type == 'industrial':
            mapping = {
                'pet': 'plastic_food_containers',
                'hdpe': 'plastic_food_containers',
                'pvc': 'plastic_food_containers',
                'ldpe': 'plastic_food_containers',
                'pp': 'plastic_food_containers',
                'ps': 'plastic_food_containers',
                'metal': 'aluminum_food_cans',
                'glass': 'glass_food_jars',
                'paper': 'office_paper',
                'cardboard': 'cardboard_boxes',
                'trash': 'food_waste'
            }
            return mapping.get(raw)

        if source_type == 'multiclass':
            mapping = {
                'plastic': 'plastic_food_containers',
                'metal': 'aluminum_food_cans',
                'glass': 'glass_food_jars',
                'paper': 'office_paper',
                'cardboard': 'cardboard_boxes',
                'trash': 'food_waste',
                'organic': 'food_waste',
                'battery': 'aerosol_cans',
                'clothes': 'clothing',
                'shoes': 'shoes'
            }
            return mapping.get(raw)

        # Universal fallback mappings for common waste categories
        # This ensures NO images are skipped
        fallback_mapping = {
            # Recyclables
            'recyclable': 'plastic_food_containers',
            'recycle': 'plastic_food_containers',
            'recycling': 'plastic_food_containers',
            # Waste types
            'waste': 'food_waste',
            'garbage': 'food_waste',
            'rubbish': 'food_waste',
            'refuse': 'food_waste',
            # Organic
            'compost': 'food_waste',
            'food': 'food_waste',
            'kitchen': 'food_waste',
            'biological': 'food_waste',
            # Paper products
            'newspaper': 'newspaper',
            'magazine': 'magazines',
            'book': 'office_paper',
            'document': 'office_paper',
            # Plastic types
            'bottle': 'plastic_water_bottles',
            'bottle-transp': 'plastic_water_bottles',
            'bottle-blue': 'plastic_water_bottles',
            'bottle-dark': 'plastic_water_bottles',
            'bottle-green': 'plastic_water_bottles',
            'bottle-blue5l': 'plastic_water_bottles',
            'bottle-milk': 'plastic_water_bottles',
            'bottle-oil': 'plastic_water_bottles',
            'bottle-yogurt': 'plastic_food_containers',
            'bottle-multicolor': 'plastic_water_bottles',
            'bottle-transp-full': 'plastic_water_bottles',
            'bottle-blue-full': 'plastic_water_bottles',
            'bottle-green-full': 'plastic_water_bottles',
            'bottle-dark-full': 'plastic_water_bottles',
            'bottle-milk-full': 'plastic_water_bottles',
            'bottle-multicolorv-full': 'plastic_water_bottles',
            'bottle-blue5l-full': 'plastic_water_bottles',
            'bottle-oil-full': 'plastic_water_bottles',
            'bag': 'plastic_shopping_bags',
            'container': 'plastic_food_containers',
            'cup': 'paper_cups',
            'straw': 'plastic_straws',
            # Detergents (plastic containers)
            'detergent-white': 'plastic_food_containers',
            'detergent-color': 'plastic_food_containers',
            'detergent-transparent': 'plastic_food_containers',
            'detergent-box': 'cardboard_boxes',
            # Metal
            'can': 'aluminum_soda_cans',
            'cans': 'aluminum_soda_cans',
            'tin': 'steel_food_cans',
            'aluminum': 'aluminum_food_cans',
            'steel': 'steel_food_cans',
            'canister': 'aluminum_food_cans',
            'battery': 'aerosol_cans',  # Hazardous, map to aerosol as closest
            # Glass
            'jar': 'glass_food_jars',
            'glass-transp': 'glass_food_jars',
            'glass-dark': 'glass_beverage_bottles',
            'glass-green': 'glass_beverage_bottles',
            'white-glass': 'glass_food_jars',
            'brown-glass': 'glass_beverage_bottles',
            'green-glass': 'glass_beverage_bottles',
            # Cardboard
            'milk-cardboard': 'cardboard_boxes',
            'juice-cardboard': 'cardboard_boxes',
            # Textiles
            'fabric': 'clothing',
            'textile': 'clothing',
            # Foam
            'foam': 'styrofoam_cups',
            'styrofoam': 'styrofoam_cups',
            'polystyrene': 'styrofoam_cups',
        }

        # Try fallback mapping
        for key, value in fallback_mapping.items():
            if key in raw:
                return value

        return None

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        """
        PRODUCTION-GRADE: Load and transform image with comprehensive error handling
        Ensures ALL images work perfectly regardless of format, size, or corruption
        """
        path, label_idx = self.samples[idx]

        try:
            # Validate path exists
            if not path.exists():
                raise FileNotFoundError(f"Image file not found: {path}")

            # Load image with PIL (handles all formats)
            # Use timeout to prevent hanging on corrupted images
            img = Image.open(path)
            img.load()  # Force load to catch corruption early

            # Convert to RGB (handles RGBA, L, P, etc.)
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # Validate image size
            if img.size[0] < 10 or img.size[1] < 10:
                raise ValueError(f"Image too small: {img.size}")

            # Apply transforms if provided
            if self.transform:
                try:
                    img = self.transform(img)
                except Exception as transform_error:
                    # Log transform errors specifically
                    logger.error(f"❌ Transform failed for {path}: {transform_error}")
                    raise
                return img, label_idx
            else:
                # Return PIL Image if no transform
                return img, label_idx

        except Exception as e:
            # CRITICAL: Log every failure - this should be RARE
            logger.warning(f"⚠️  Failed to load image {idx}: {path}")
            logger.warning(f"   Error: {type(e).__name__}: {e}")

            # Track failure count
            if not hasattr(self, '_failure_count'):
                self._failure_count = 0
            self._failure_count += 1

            # ABORT if too many failures (>1% of dataset)
            max_failures = max(100, len(self.samples) // 100)
            if self._failure_count > max_failures:
                raise RuntimeError(
                    f"❌ TOO MANY IMAGE LOADING FAILURES ({self._failure_count}/{len(self.samples)})! "
                    f"This indicates a serious dataset problem. Aborting."
                )

            # FALLBACK: Return a valid dummy image (PIL or tensor depending on transform)
            # This should only happen for truly corrupt images (< 1%)
            if self.transform:
                # Return dummy tensor if transform is expected
                dummy_tensor = torch.zeros(3, 224, 224, dtype=torch.float32)
                # Apply CLIP normalization to match real images
                dummy_tensor[0] = -0.48145466 / 0.26862954  # Normalized black for R
                dummy_tensor[1] = -0.4578275 / 0.26130258   # Normalized black for G
                dummy_tensor[2] = -0.40821073 / 0.27577711  # Normalized black for B
                return dummy_tensor, label_idx
            else:
                # Return dummy PIL image if no transform
                dummy_img = Image.new('RGB', (224, 224), color=(0, 0, 0))
                return dummy_img, label_idx

    def get_labels(self):
        return [s[1] for s in self.samples]


In [ ]:
class TransformSubset(torch.utils.data.Dataset):
    """
    CRITICAL FIX: Wrapper for Subset that properly applies transforms

    Problem: torch.utils.data.random_split creates Subset objects that share
    the same underlying dataset. Setting dataset.transform affects BOTH train
    and validation, causing catastrophic failures.

    Solution: This wrapper applies transforms independently for each subset.
    """
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self._load_count = 0  # Track how many items loaded

    def __getitem__(self, idx):
        try:
            # Get item from underlying subset (without transform)
            img, label = self.subset[idx]

            # Apply our transform
            if self.transform:
                img = self.transform(img)

            # Track successful loads (for debugging)
            self._load_count += 1
            if self._load_count % 100 == 0:
                logger.debug(f"Loaded {self._load_count} samples successfully")

            return img, label

        except Exception as e:
            logger.error(f"❌ TransformSubset failed at idx {idx}: {e}")
            raise

    def __len__(self):
        return len(self.subset)


In [ ]:
class ManualTransform:
    """
    PRODUCTION-GRADE: Manual transform that bypasses NumPy entirely.
    Fixes NumPy 2.x incompatibility with PyTorch 2.x.

    This transform:
    1. Takes PIL Image as input
    2. Performs all operations using PIL and PyTorch only
    3. Returns PyTorch tensor
    4. NO NumPy involved at any step
    """
    def __init__(self, input_size=224, mean=(0.48145466, 0.4578275, 0.40821073),
                 std=(0.26862954, 0.26130258, 0.27577711), is_train=True):
        self.input_size = input_size
        self.mean = torch.tensor(mean).view(3, 1, 1)
        self.std = torch.tensor(std).view(3, 1, 1)
        self.is_train = is_train

    def __call__(self, img):
        """
        Transform PIL Image to normalized tensor.

        Args:
            img: PIL Image (RGB)

        Returns:
            torch.Tensor: Normalized image tensor (3, H, W)
        """
        # Ensure RGB mode
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # ULTRA-OPTIMIZED: MINIMAL augmentation for maximum initial accuracy
        if self.is_train:
            # Random horizontal flip ONLY (50% probability)
            # This is the ONLY augmentation that doesn't distort waste object features
            if random.random() > 0.5:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)

        # Resize to target size
        img = img.resize((self.input_size, self.input_size), Image.BICUBIC)

        # Convert PIL Image to tensor WITHOUT NumPy
        # Method: Use img.tobytes() -> torch.frombuffer()
        img_bytes = img.tobytes()
        img_tensor = torch.frombuffer(img_bytes, dtype=torch.uint8).clone()
        img_tensor = img_tensor.view(self.input_size, self.input_size, 3)

        # Convert HWC to CHW
        img_tensor = img_tensor.permute(2, 0, 1).contiguous()

        # Convert to float and normalize to [0, 1]
        img_tensor = img_tensor.float().div_(255.0)

        # Apply ImageNet normalization
        img_tensor = img_tensor.sub_(self.mean).div_(self.std)

        return img_tensor

In [ ]:
def get_vision_transforms(config, model, is_train=True):
    """
    PRODUCTION-GRADE: Get vision transforms using MANUAL transforms (NO NumPy).
    Fixes NumPy 2.x incompatibility with PyTorch 2.x.

    This completely bypasses torchvision.transforms.ToTensor() which uses NumPy.
    """
    # Get config's input_size (prioritize config over model defaults)
    config_input_size = config.get('data', {}).get('input_size', 224)

    # Get model config for input size and normalization
    try:
        if hasattr(model, 'default_cfg'):
            model_cfg = model.default_cfg
        elif hasattr(model, 'pretrained_cfg'):
            model_cfg = model.pretrained_cfg
        else:
            model_cfg = {
                'input_size': (3, config_input_size, config_input_size),
                'mean': (0.48145466, 0.4578275, 0.40821073),  # CLIP/EVA02 normalization
                'std': (0.26862954, 0.26130258, 0.27577711)
            }

        # Extract input size (handle tuple format)
        input_size_tuple = model_cfg.get('input_size', (3, config_input_size, config_input_size))
        if isinstance(input_size_tuple, tuple) and len(input_size_tuple) == 3:
            img_size = input_size_tuple[1]  # Get height (assuming square images)
        else:
            img_size = config_input_size  # Use config's input_size

        mean = model_cfg.get('mean', (0.48145466, 0.4578275, 0.40821073))
        std = model_cfg.get('std', (0.26862954, 0.26130258, 0.27577711))

    except Exception as e:
        logger.warning(f"Failed to get model config: {e}, using defaults")
        # Force config's input_size for Mac compatibility (override model default)
        img_size = config_input_size
        mean = (0.48145466, 0.4578275, 0.40821073)
        std = (0.26862954, 0.26130258, 0.27577711)

    logger.info(f"✅ Using MANUAL transforms (NumPy-free) with input_size={img_size}, mean={mean}, std={std}")

    # Use ManualTransform that bypasses NumPy entirely
    return ManualTransform(
        input_size=img_size,
        mean=mean,
        std=std,
        is_train=is_train
    )

In [ ]:
def create_vision_model(config):
    logger.info(f"Creating model: {config['model']['backbone']}")
    logger.info(f"Pretrained: {config['model']['pretrained']}")

    model = timm.create_model(
        config["model"]["backbone"],
        pretrained=config["model"]["pretrained"],
        num_classes=config["model"]["num_classes"],
        drop_rate=config["model"]["drop_rate"],
        drop_path_rate=config["model"]["drop_path_rate"]
    )

    # CRITICAL: Verify pretrained weights loaded
    if config["model"]["pretrained"]:
        logger.info("✅ Pretrained weights loaded successfully")

        # Check if model has reasonable initial weights (not random)
        first_param = next(model.parameters())
        param_mean = first_param.data.mean().item()
        param_std = first_param.data.std().item()
        logger.info(f"   First layer stats: mean={param_mean:.6f}, std={param_std:.6f}")

        # Pretrained weights should have non-zero mean and reasonable std
        if abs(param_mean) < 1e-6 and abs(param_std) < 1e-6:
            logger.warning("⚠️  WARNING: Weights look like zeros! Pretrained loading may have failed!")
        elif abs(param_mean) > 10 or param_std > 10:
            logger.warning("⚠️  WARNING: Weights have extreme values! Check pretrained loading!")
        else:
            logger.info("   ✅ Weight statistics look good (pretrained weights confirmed)")

        # ULTRA-OPTIMIZED: Ensure ALL layers are trainable (no freezing)
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        logger.info(f"   Trainable: {trainable_params/1e6:.2f}M / {total_params/1e6:.2f}M parameters")

        if trainable_params < total_params:
            logger.warning(f"   ⚠️  Some layers are frozen! Unfreezing all layers for maximum performance...")
            for param in model.parameters():
                param.requires_grad = True
            logger.info(f"   ✅ All {total_params/1e6:.2f}M parameters are now trainable")
    else:
        logger.warning("⚠️  Training from scratch (no pretrained weights)")

    # CRITICAL: Verify model output dimensions
    logger.info("🔍 Verifying model output dimensions...")
    model.eval()
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, 224, 224)
        dummy_output = model(dummy_input)
        logger.info(f"   Model output shape: {dummy_output.shape}")
        logger.info(f"   Expected: torch.Size([1, {config['model']['num_classes']}])")

        if dummy_output.shape[1] != config['model']['num_classes']:
            raise ValueError(
                f"Model output dimension mismatch! "
                f"Got {dummy_output.shape[1]}, expected {config['model']['num_classes']}"
            )
        logger.info("   ✅ Model output dimensions correct")
    model.train()

    return model

In [ ]:
def train_vision_model(config, resume_from_checkpoint=None):
    """
    Professional-grade vision model training with comprehensive error handling.
    Optimized for Tesla T4 GPU (14.74 GB) with production-ready memory management.

    Args:
        config: Training configuration dictionary
        resume_from_checkpoint: Path to checkpoint file to resume from (optional)

    Returns:
        Trained model or None if training fails
    """
    start_epoch = 0
    best_val_acc = 0.0
    metrics_history = {
        "train_loss": [], "train_acc": [],
        "val_loss": [], "val_acc": [],
        "per_class_f1": [], "learning_rate": []
    }

    try:
        set_seed()
        device = get_device()
        optimize_memory(device)
        logger.info(f"Using device: {device}")

        # Create and configure model
        model = create_vision_model(config).to(device)
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        logger.info(f"Model parameters: {total_params / 1e6:.2f}M total, {trainable_params / 1e6:.2f}M trainable")

        # Enable gradient checkpointing for memory efficiency (NOT compatible with MPS)
        if device.type != "mps" and hasattr(model, 'set_grad_checkpointing'):
            model.set_grad_checkpointing(enable=True)
            logger.info("✓ Gradient checkpointing enabled (saves ~40% memory)")
        elif device.type == "mps":
            logger.warning("⚠️  Gradient checkpointing disabled (incompatible with MPS - causes crashes)")

    except Exception as e:
        logger.error(f"Model initialization failed: {e}")
        raise

    train_transform = get_vision_transforms(config, model, is_train=True)
    val_transform = get_vision_transforms(config, model, is_train=False)

    # CRITICAL: Validate transform output size matches model expectations
    logger.info("🔍 Validating transform pipeline...")
    try:
        # Create dummy image for validation
        import numpy as np
        dummy_img = Image.new('RGB', (640, 480), color=(128, 128, 128))

        # Convert to numpy array first to avoid NumPy version compatibility issues
        # This works around the "expected np.ndarray (got numpy.ndarray)" error
        dummy_array = np.array(dummy_img)
        dummy_img_fixed = Image.fromarray(dummy_array)

        transformed = train_transform(dummy_img_fixed)
        actual_size = transformed.shape
        expected_channels = 3
        expected_size = config.get('data', {}).get('input_size', 224)

        logger.info(f"  Transform output shape: {actual_size}")
        logger.info(f"  Expected: ({expected_channels}, {expected_size}, {expected_size})")

        if actual_size[0] != expected_channels or actual_size[1] != expected_size or actual_size[2] != expected_size:
            raise ValueError(
                f"Transform size mismatch! Output {actual_size} != Expected ({expected_channels}, {expected_size}, {expected_size})"
            )
        logger.info("  ✅ Transform validation passed")
    except TypeError as e:
        if "expected np.ndarray" in str(e):
            logger.warning(f"⚠️  NumPy compatibility issue detected: {e}")
            logger.warning("  Skipping transform validation (will validate with real data)")
            logger.warning("  Consider upgrading: pip install 'numpy<2.0'")
        else:
            logger.error(f"❌ Transform validation failed: {e}")
            raise
    except Exception as e:
        logger.error(f"❌ Transform validation failed: {e}")
        raise

    full_dataset = UnifiedWasteDataset(
        sources_config=config["data"]["sources"],
        target_classes=TARGET_CLASSES,
        transform=None
    )

    if len(full_dataset) == 0:
        logger.error("Dataset is empty. Check paths.")
        return None

    # CRITICAL: Validate that samples actually exist on disk
    logger.info("🔍 Validating dataset samples...")
    missing_count = 0
    sample_check_count = min(1000, len(full_dataset.samples))

    for i in range(sample_check_count):
        path, label = full_dataset.samples[i]
        if not path.exists():
            missing_count += 1
            if missing_count <= 5:  # Log first 5 missing files
                logger.error(f"  ❌ Missing file: {path}")

    if missing_count > 0:
        logger.error(f"❌ {missing_count}/{sample_check_count} sample files are MISSING!")
        raise FileNotFoundError(
            f"{missing_count} image files are missing from disk. "
            "Dataset paths may be incorrect."
        )

    logger.info(f"  ✅ All {sample_check_count} checked samples exist on disk")

    # CRITICAL FIX: Create train/val split WITHOUT transforms first
    train_size = int(0.85 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

    logger.info(f"📊 Dataset split: {train_size} train, {val_size} validation")

    # CRITICAL FIX: Wrap subsets with independent transforms
    # This ensures train and val use DIFFERENT transforms (not shared!)
    train_dataset = TransformSubset(train_subset, transform=train_transform)
    val_dataset = TransformSubset(val_subset, transform=val_transform)

    logger.info("✅ Train dataset: using training transforms (with augmentation)")
    logger.info("✅ Val dataset: using validation transforms (NO augmentation)")

    train_loader = DataLoader(
        train_dataset,
        batch_size=config["training"]["batch_size"],
        shuffle=True,
        num_workers=0,  # Disabled for macOS compatibility (multiprocessing issue)
        pin_memory=config["data"]["pin_memory"],
        persistent_workers=False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config["training"]["batch_size"] * 2,
        shuffle=False,
        num_workers=0,  # Disabled for macOS compatibility (multiprocessing issue)
        persistent_workers=False
    )

    logger.info(f"✅ Train loader: {len(train_loader)} batches")
    logger.info(f"✅ Val loader: {len(val_loader)} batches")

    # CRITICAL: Test single sample first to catch issues early
    logger.info("🔍 Testing single sample load...")
    try:
        test_img, test_label = train_dataset[0]
        logger.info(f"  ✅ Single sample loaded: shape={test_img.shape}, label={test_label}")
    except Exception as e:
        logger.error(f"❌ Failed to load single sample: {e}")
        logger.exception(e)
        raise RuntimeError("Cannot load even a single sample! Check dataset and transforms.")

    # CRITICAL: Verify data loading and label distribution
    logger.info("🔍 Verifying data loading and label distribution...")
    logger.info("   Loading first batch (this may take 10-30 seconds)...")
    try:
        # Test train loader
        logger.info("   Creating train batch iterator...")
        train_batch_iter = iter(train_loader)
        logger.info("   Loading first train batch (32 images)...")
        train_images, train_labels = next(train_batch_iter)
        logger.info(f"  ✅ Train batch loaded successfully!")
        logger.info(f"     Shape: {train_images.shape}, labels: {train_labels.shape}")
        logger.info(f"     Label range: [{train_labels.min().item()}, {train_labels.max().item()}]")
        logger.info(f"     Unique labels in batch: {len(torch.unique(train_labels))}")

        # Test val loader
        logger.info("   Loading first val batch...")
        val_batch_iter = iter(val_loader)
        val_images, val_labels = next(val_batch_iter)
        logger.info(f"  ✅ Val batch loaded successfully!")
        logger.info(f"     Shape: {val_images.shape}, labels: {val_labels.shape}")
        logger.info(f"     Label range: [{val_labels.min().item()}, {val_labels.max().item()}]")
        logger.info(f"     Unique labels in batch: {len(torch.unique(val_labels))}")

        # Verify labels are in valid range [0, 29]
        if train_labels.min() < 0 or train_labels.max() >= 30:
            raise ValueError(f"Train labels out of range! Min: {train_labels.min()}, Max: {train_labels.max()}")
        if val_labels.min() < 0 or val_labels.max() >= 30:
            raise ValueError(f"Val labels out of range! Min: {val_labels.min()}, Max: {val_labels.max()}")

        logger.info("  ✅ All labels are in valid range [0, 29]")
        logger.info("  ✅ Data loading verification PASSED!")

    except Exception as e:
        logger.error(f"❌ Data loading verification failed: {e}")
        logger.exception(e)  # Full traceback
        raise

    # PEAK OPTIMIZER: AdamW with optimal hyperparameters
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config["training"]["learning_rate"],
        weight_decay=config["training"]["weight_decay"],
        betas=(0.9, 0.999),  # Standard momentum
        eps=1e-8
    )

    # ULTRA-OPTIMIZED CRITERION: NO label smoothing for maximum accuracy
    criterion = nn.CrossEntropyLoss(label_smoothing=0.0)  # Clear signal for pretrained model

    # Professional training configuration
    use_amp = config["training"].get("use_amp", False) and (device.type == "cuda")
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp) if use_amp else None
    accumulation_steps = config["training"]["grad_accum_steps"]
    max_grad_norm = config["training"].get("max_grad_norm", 1.0)
    warmup_epochs = config["training"].get("warmup_epochs", 0)

    # PEAK SCHEDULER: Cosine annealing with warmup for optimal convergence
    total_steps = len(train_loader) * config["training"]["num_epochs"] // accumulation_steps
    warmup_steps = (len(train_loader) * warmup_epochs) // accumulation_steps

    if warmup_steps > 0:
        # ULTRA-OPTIMIZED: OneCycleLR with AGGRESSIVE warmup for pretrained model
        scheduler = optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=config["training"]["learning_rate"],  # Peak at 1e-3
            total_steps=total_steps,
            pct_start=warmup_steps / total_steps,  # Warmup percentage
            anneal_strategy='cos',
            div_factor=2.0,  # Start at LR/2 (5e-4) - VERY AGGRESSIVE for pretrained
            final_div_factor=20.0  # End at LR/20 (5e-5) - Good final convergence
        )
    else:
        # Use CosineAnnealingLR without warmup
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=total_steps,
            eta_min=config["training"]["learning_rate"] / 20  # Good final LR
        )

    # CRITICAL: Load checkpoint if resuming training
    if resume_from_checkpoint and Path(resume_from_checkpoint).exists():
        logger.info(f"📂 Loading checkpoint from {resume_from_checkpoint}")
        checkpoint = torch.load(resume_from_checkpoint, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        if 'scheduler_state_dict' in checkpoint:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        start_epoch = checkpoint.get('epoch', 0) + 1  # Start from next epoch
        best_val_acc = checkpoint.get('val_acc', 0.0)

        if 'metrics_history' in checkpoint:
            metrics_history = checkpoint['metrics_history']

        logger.info(f"✅ Resumed from epoch {start_epoch}, best val acc: {best_val_acc:.2f}%")
    elif resume_from_checkpoint:
        logger.warning(f"⚠️  Checkpoint not found: {resume_from_checkpoint}. Starting from scratch.")

    early_stopping = EarlyStopping(patience=config["training"]["patience"])

    if device.type == "mps":
        logger.info("ℹ️  MPS detected: AMP disabled (not supported on Apple Silicon)")
    elif device.type == "cpu":
        logger.info("ℹ️  CPU detected: AMP disabled (not supported on CPU)")
    else:
        logger.info(f"ℹ️  AMP {'enabled' if use_amp else 'disabled'}")

    logger.info(f"Training configuration:")
    logger.info(f"  - Batch size: {config['training']['batch_size']}")
    logger.info(f"  - Gradient accumulation: {accumulation_steps}")
    logger.info(f"  - Effective batch size: {config['training']['batch_size'] * accumulation_steps}")
    logger.info(f"  - Mixed precision (AMP): {use_amp}")
    logger.info(f"  - Gradient clipping: {max_grad_norm}")
    logger.info(f"  - Learning rate: {config['training']['learning_rate']}")

    # INDUSTRIAL-GRADE: Best model tracking
    # Note: best_val_acc and metrics_history initialized at function start (for checkpoint resume support)
    best_model_state = None
    checkpoint_dir = Path("checkpoints")
    checkpoint_dir.mkdir(exist_ok=True)

    # Initialize Weights & Biases with graceful fallback
    try:
        wandb.init(project="sustainability-vision-lake", config=config, mode="online")
        logger.info("✓ W&B logging enabled")
    except Exception as e:
        logger.warning(f"W&B initialization failed: {e}. Continuing without logging.")
        wandb.init(mode="disabled")

    # CRITICAL: Pre-training sanity check - validate one batch
    logger.info("🔍 Running pre-training sanity check...")
    try:
        model.eval()
        test_batch = next(iter(train_loader))
        test_images, test_labels = test_batch

        logger.info(f"  Batch shape: {test_images.shape}")
        logger.info(f"  Expected: [batch_size, 3, {config.get('data', {}).get('input_size', 224)}, {config.get('data', {}).get('input_size', 224)}]")

        # Validate batch dimensions
        expected_size = config.get('data', {}).get('input_size', 224)
        if test_images.shape[1] != 3:
            raise ValueError(f"Invalid channel count: {test_images.shape[1]} (expected 3)")
        if test_images.shape[2] != expected_size or test_images.shape[3] != expected_size:
            raise ValueError(
                f"Invalid image size: {test_images.shape[2]}x{test_images.shape[3]} "
                f"(expected {expected_size}x{expected_size})"
            )

        # Test forward pass
        use_non_blocking = (device.type == "cuda")
        test_images = test_images.to(device, non_blocking=use_non_blocking)
        test_labels = test_labels.to(device, non_blocking=use_non_blocking)

        with torch.no_grad():
            test_outputs = model(test_images)
            logger.info(f"  Model output shape: {test_outputs.shape}")
            logger.info(f"  Expected: [batch_size, {config['model']['num_classes']}]")

            if test_outputs.shape[1] != config['model']['num_classes']:
                raise ValueError(
                    f"Invalid output classes: {test_outputs.shape[1]} "
                    f"(expected {config['model']['num_classes']})"
                )

        logger.info("  ✅ Pre-training sanity check passed!")
        logger.info(f"  ✅ All images are {expected_size}x{expected_size}")
        logger.info(f"  ✅ Model accepts input and produces correct output shape")

        # Clean up
        del test_batch, test_images, test_labels, test_outputs
        if device.type == "cuda":
            torch.cuda.empty_cache()
        elif device.type == "mps":
            try:
                torch.mps.empty_cache()
            except AttributeError:
                pass

    except Exception as e:
        logger.error(f"❌ Pre-training sanity check failed: {e}")
        logger.error("This indicates a fundamental configuration issue. Aborting training.")
        raise

    # COMPREHENSIVE DATA QUALITY VALIDATION
    logger.info("="*80)
    logger.info("🔬 COMPREHENSIVE DATA QUALITY VALIDATION")
    logger.info("="*80)

    try:
        # CRITICAL TEST 1: Validate individual image loading
        logger.info("Test 1: Individual Image Loading (100 random samples)...")
        import random
        sample_indices = random.sample(range(len(train_dataset)), min(100, len(train_dataset)))

        failed_samples = []
        dummy_tensor_count = 0

        for idx in sample_indices:
            try:
                img, label = train_dataset[idx]

                # Check if this is a dummy tensor (all zeros after normalization)
                if isinstance(img, torch.Tensor):
                    # Check if it's the dummy tensor pattern
                    expected_dummy_r = -0.485 / 0.229
                    expected_dummy_g = -0.456 / 0.224
                    expected_dummy_b = -0.406 / 0.225

                    if (torch.allclose(img[0], torch.full_like(img[0], expected_dummy_r), atol=1e-4) and
                        torch.allclose(img[1], torch.full_like(img[1], expected_dummy_g), atol=1e-4) and
                        torch.allclose(img[2], torch.full_like(img[2], expected_dummy_b), atol=1e-4)):
                        dummy_tensor_count += 1
                        failed_samples.append(idx)

            except Exception as e:
                failed_samples.append(idx)
                logger.error(f"  ❌ Sample {idx} failed: {e}")

        success_rate = (len(sample_indices) - len(failed_samples)) / len(sample_indices) * 100
        logger.info(f"  ✅ Success rate: {success_rate:.1f}% ({len(sample_indices) - len(failed_samples)}/{len(sample_indices)})")
        logger.info(f"  ⚠️  Dummy tensors detected: {dummy_tensor_count}")
        logger.info(f"  ⚠️  Failed samples: {len(failed_samples)}")

        # ABORT if too many failures
        if dummy_tensor_count > 5:
            logger.error(f"❌ TOO MANY DUMMY TENSORS ({dummy_tensor_count}/100)!")
            logger.error("   This means images are NOT loading correctly.")
            logger.error("   Checking first failed sample for details...")

            if failed_samples:
                # Get details about first failure
                first_fail_idx = failed_samples[0]
                path, label = train_dataset.dataset.samples[first_fail_idx]
                logger.error(f"   Failed sample path: {path}")
                logger.error(f"   Path exists: {path.exists()}")
                if path.exists():
                    logger.error(f"   File size: {path.stat().st_size} bytes")

            raise RuntimeError(
                f"Dataset loading is BROKEN! {dummy_tensor_count}/100 samples are dummy tensors. "
                "This is NOT acceptable for training."
            )

        # CRITICAL TEST 2: Validate batch loading
        logger.info("Test 2: Batch Loading (10 random batches)...")
        batch_count = 0
        total_images = 0
        value_ranges = []

        for batch_idx, (images, labels) in enumerate(train_loader):
            if batch_idx >= 10:
                break

            batch_count += 1
            total_images += images.size(0)

            # Check for NaN or Inf
            if torch.isnan(images).any():
                logger.error(f"  ❌ Batch {batch_idx}: Contains NaN values!")
                raise ValueError("Dataset contains NaN values")

            if torch.isinf(images).any():
                logger.error(f"  ❌ Batch {batch_idx}: Contains Inf values!")
                raise ValueError("Dataset contains Inf values")

            # Track value ranges
            batch_min = images.min().item()
            batch_max = images.max().item()
            value_ranges.append((batch_min, batch_max))

            # Check if batch contains mostly dummy tensors
            # Dummy tensors have very specific values
            if batch_min > -2.5 and batch_max < -1.5:
                logger.warning(f"  ⚠️  Batch {batch_idx} may contain dummy tensors (range: [{batch_min:.3f}, {batch_max:.3f}])")

            # Validate labels
            if labels.min() < 0 or labels.max() >= 30:
                logger.error(f"  ❌ Batch {batch_idx}: Invalid labels [{labels.min()}, {labels.max()}]")
                raise ValueError(f"Invalid label range: [{labels.min()}, {labels.max()}]")

        # Report statistics
        min_vals = [r[0] for r in value_ranges]
        max_vals = [r[1] for r in value_ranges]
        overall_min = min(min_vals)
        overall_max = max(max_vals)

        logger.info(f"  ✅ Tested {batch_count} batches ({total_images} images)")
        logger.info(f"  ✅ Value range: [{overall_min:.3f}, {overall_max:.3f}]")
        logger.info(f"  ✅ No NaN or Inf values detected")
        logger.info(f"  ✅ All labels valid [0, 29]")

        # Validate value range is reasonable for real images
        # Real images after ImageNet normalization should be roughly [-2.5, 2.5]
        if overall_min > -1.0 or overall_max < 1.0:
            logger.warning(f"  ⚠️  WARNING: Value range [{overall_min:.3f}, {overall_max:.3f}] seems narrow!")
            logger.warning(f"     Expected range for real images: approximately [-2.5, 2.5]")
            logger.warning(f"     This may indicate dummy tensors are being used!")

    except Exception as e:
        logger.error(f"❌ Data quality validation failed: {e}")
        raise

    logger.info("="*80)
    logger.info("✅ ALL VALIDATION TESTS PASSED - READY TO TRAIN")
    logger.info("="*80)

    model.train()  # Set back to training mode

    # Main training loop with error handling
    try:
        for epoch in range(start_epoch, config["training"]["num_epochs"]):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0

            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['training']['num_epochs']}")
            optimizer.zero_grad()

            for i, (images, labels) in enumerate(pbar):
                try:
                    # CRITICAL: Aggressive memory cleanup every 100 batches for MPS stability
                    if i > 0 and i % 100 == 0 and device.type == "mps":
                        try:
                            torch.mps.empty_cache()
                        except AttributeError:
                            pass

                    # CRITICAL: Validate batch shape before processing
                    expected_size = config.get('data', {}).get('input_size', 224)
                    if images.shape[1] != 3 or images.shape[2] != expected_size or images.shape[3] != expected_size:
                        logger.error(
                            f"❌ Batch {i} has invalid shape: {images.shape}. "
                            f"Expected: [batch_size, 3, {expected_size}, {expected_size}]. Skipping batch."
                        )
                        continue

                    # non_blocking only works with CUDA + pin_memory
                    use_non_blocking = (device.type == "cuda")
                    images, labels = images.to(device, non_blocking=use_non_blocking), labels.to(device, non_blocking=use_non_blocking)

                except RuntimeError as e:
                    if "out of memory" in str(e) or "MPS" in str(e):
                        logger.error(f"OOM at batch {i}. Clearing cache and skipping batch.")
                        if device.type == "cuda":
                            torch.cuda.empty_cache()
                        elif device.type == "mps":
                            try:
                                torch.mps.empty_cache()
                            except AttributeError:
                                pass  # MPS empty_cache not available in older PyTorch
                        continue
                    raise
                except AssertionError as e:
                    # Catch assertion errors from model (e.g., size mismatches)
                    logger.error(f"❌ Assertion error at batch {i}: {e}")
                    logger.error(f"   Batch shape: {images.shape}")
                    logger.error(f"   This indicates a size mismatch. Skipping batch.")
                    continue

                try:
                    if use_amp:
                        with torch.cuda.amp.autocast():
                            outputs = model(images)
                            loss = criterion(outputs, labels) / accumulation_steps
                        scaler.scale(loss).backward()

                        if (i + 1) % accumulation_steps == 0:
                            scaler.unscale_(optimizer)

                            # CRITICAL: Check gradient health
                            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
                            if not np.isfinite(grad_norm.item()):
                                logger.error(f"❌ Non-finite gradient norm at batch {i}: {grad_norm.item()}")
                                logger.error("   Resetting optimizer state and skipping update.")
                                optimizer.zero_grad()
                                continue

                            scaler.step(optimizer)
                            scaler.update()
                            optimizer.zero_grad()
                            scheduler.step()  # OneCycleLR steps per batch
                    else:
                        outputs = model(images)
                        loss = criterion(outputs, labels) / accumulation_steps
                        loss.backward()
                        if (i + 1) % accumulation_steps == 0:
                            # CRITICAL: Check gradient health
                            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
                            if not np.isfinite(grad_norm.item()):
                                logger.error(f"❌ Non-finite gradient norm at batch {i}: {grad_norm.item()}")
                                logger.error("   Resetting optimizer state and skipping update.")
                                optimizer.zero_grad()
                                continue

                            optimizer.step()
                            optimizer.zero_grad()
                            scheduler.step()  # OneCycleLR steps per batch

                except AssertionError as e:
                    # Catch assertion errors from model forward pass
                    logger.error(f"❌ Model forward pass failed at batch {i}: {e}")
                    logger.error(f"   Input shape: {images.shape}")
                    logger.error(f"   Expected size: {expected_size}x{expected_size}")
                    logger.error("   Skipping batch and continuing training.")
                    continue

                # CRITICAL: Check for NaN/Inf in loss
                loss_value = loss.item() * accumulation_steps
                if not np.isfinite(loss_value):
                    logger.error(f"❌ Non-finite loss detected at batch {i}: {loss_value}")
                    logger.error("   This indicates training instability. Skipping batch.")
                    # Reset gradients to prevent corruption
                    optimizer.zero_grad()
                    continue

                running_loss += loss_value
                with torch.no_grad():
                    _, predicted = outputs.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()

                    # CRITICAL DEBUG: Log first batch predictions
                    if i == 0:
                        logger.info(f"🔍 FIRST TRAINING BATCH:")
                        logger.info(f"   Images shape: {images.shape}")
                        logger.info(f"   Images min/max: {images.min().item():.3f} / {images.max().item():.3f}")
                        logger.info(f"   Labels: {labels[:10].cpu().numpy()}")
                        logger.info(f"   Predictions: {predicted[:10].cpu().numpy()}")
                        logger.info(f"   Correct: {predicted.eq(labels).sum().item()}/{labels.size(0)}")
                        logger.info(f"   Output logits (first sample): {outputs[0, :10].cpu().numpy()}")

                current_loss = running_loss / (i + 1)
                pbar.set_postfix({'loss': f"{current_loss:.4f}", 'acc': f"{100*correct/total:.2f}%"})

            train_acc = 100 * correct / total

            # CRITICAL: Epoch-level health check
            if total == 0:
                logger.error(f"❌ Epoch {epoch+1}: No samples processed! This indicates a critical data loading issue.")
                break

            avg_train_loss = running_loss / len(train_loader) if len(train_loader) > 0 else float('inf')
            if not np.isfinite(avg_train_loss):
                logger.error(f"❌ Epoch {epoch+1}: Non-finite training loss: {avg_train_loss}")
                logger.error("   Training has become unstable. Stopping.")
                break

            # INDUSTRIAL-GRADE: Comprehensive validation with per-class metrics
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            all_preds = []
            all_labels = []

            # CRITICAL: Log first batch details for debugging
            first_batch_logged = False

            with torch.no_grad():
                for val_i, (images, labels) in enumerate(tqdm(val_loader, desc="Validation", leave=False)):
                    try:
                        # CRITICAL: Log first batch for debugging
                        if not first_batch_logged:
                            logger.info(f"🔍 First validation batch:")
                            logger.info(f"   Images shape: {images.shape}")
                            logger.info(f"   Labels shape: {labels.shape}")
                            logger.info(f"   Labels range: [{labels.min().item()}, {labels.max().item()}]")
                            logger.info(f"   Unique labels: {len(torch.unique(labels))}")
                            first_batch_logged = True

                        # Validate batch shape
                        expected_size = config.get('data', {}).get('input_size', 224)
                        if images.shape[1] != 3 or images.shape[2] != expected_size or images.shape[3] != expected_size:
                            logger.warning(
                                f"⚠️  Validation batch {val_i} has invalid shape: {images.shape}. Skipping."
                            )
                            continue

                        use_non_blocking = (device.type == "cuda")
                        images, labels = images.to(device, non_blocking=use_non_blocking), labels.to(device, non_blocking=use_non_blocking)

                        if use_amp:
                            with torch.cuda.amp.autocast():
                                outputs = model(images)
                                loss = criterion(outputs, labels)
                        else:
                            outputs = model(images)
                            loss = criterion(outputs, labels)

                        # CRITICAL: Log first batch predictions
                        if val_i == 0:
                            _, predicted = outputs.max(1)
                            logger.info(f"   First batch predictions: {predicted[:10].cpu().numpy()}")
                            logger.info(f"   First batch ground truth: {labels[:10].cpu().numpy()}")
                            logger.info(f"   First batch correct: {predicted.eq(labels).sum().item()}/{labels.size(0)}")

                        loss_val = loss.item()
                        if not np.isfinite(loss_val):
                            logger.warning(f"⚠️  Non-finite validation loss at batch {val_i}: {loss_val}. Skipping.")
                            continue

                        val_loss += loss_val
                        _, predicted = outputs.max(1)
                        val_total += labels.size(0)
                        val_correct += predicted.eq(labels).sum().item()

                        # Collect for per-class metrics
                        all_preds.extend(predicted.cpu().numpy())
                        all_labels.extend(labels.cpu().numpy())

                    except Exception as e:
                        logger.warning(f"⚠️  Validation batch {val_i} failed: {e}. Skipping.")
                        logger.exception(e)  # Full traceback
                        continue

            # CRITICAL: Validation health check
            if val_total == 0:
                logger.error(f"❌ Epoch {epoch+1}: No validation samples processed!")
                logger.error("   This indicates a critical issue with validation data.")
                logger.error(f"   Val loader length: {len(val_loader)}")
                logger.error(f"   Val dataset length: {len(val_dataset)}")
                break

            logger.info(f"📊 Validation summary: {val_correct}/{val_total} correct ({100*val_correct/val_total:.2f}%)")

            val_acc = 100 * val_correct / val_total
            val_loss /= len(val_loader)

            if not np.isfinite(val_loss):
                logger.error(f"❌ Epoch {epoch+1}: Non-finite validation loss: {val_loss}")
                logger.error("   Validation has become unstable. Stopping.")
                break

            # INDUSTRIAL-GRADE: Per-class metrics
            if len(all_preds) == 0 or len(all_labels) == 0:
                logger.error(f"❌ Epoch {epoch+1}: No predictions collected for metrics!")
                break

            precision, recall, f1, support = precision_recall_fscore_support(
                all_labels, all_preds, average=None, zero_division=0
            )
            macro_f1 = f1.mean()

            # Find worst performing classes
            worst_classes_idx = np.argsort(f1)[:5]
            logger.info(f"📊 Per-Class Performance:")
            logger.info(f"  Macro F1: {macro_f1:.4f}")
            logger.info(f"  Worst 5 classes:")
            for idx in worst_classes_idx:
                if idx < len(TARGET_CLASSES):
                    logger.info(f"    {TARGET_CLASSES[idx]}: F1={f1[idx]:.4f}, Support={support[idx]}")

            logger.info(f"Epoch {epoch+1}/{config['training']['num_epochs']}: Train Acc {train_acc:.2f}%, Val Loss {val_loss:.4f}, Val Acc {val_acc:.2f}%, Macro F1 {macro_f1:.4f}")

            # INDUSTRIAL-GRADE: Track metrics history
            metrics_history["train_acc"].append(train_acc)
            metrics_history["val_acc"].append(val_acc)
            metrics_history["val_loss"].append(val_loss)
            metrics_history["per_class_f1"].append(macro_f1)
            metrics_history["learning_rate"].append(optimizer.param_groups[0]['lr'])

            try:
                wandb.log({
                    "epoch": epoch + 1,
                    "train_acc": train_acc,
                    "val_acc": val_acc,
                    "val_loss": val_loss,
                    "macro_f1": macro_f1,
                    "learning_rate": optimizer.param_groups[0]['lr'],
                    "worst_class_f1": f1[worst_classes_idx[0]] if len(worst_classes_idx) > 0 else 0
                })
            except:
                pass

            # INDUSTRIAL-GRADE: Save best model checkpoint
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_state = {
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'val_acc': val_acc,
                    'val_loss': val_loss,
                    'macro_f1': macro_f1,
                    'config': config,
                    'metrics_history': metrics_history
                }
                checkpoint_path = checkpoint_dir / f"best_model_epoch{epoch+1}_acc{val_acc:.2f}.pth"
                torch.save(best_model_state, checkpoint_path)
                logger.info(f"✓ Saved best model checkpoint: {checkpoint_path}")

                # Keep only best checkpoint, delete others
                for old_ckpt in checkpoint_dir.glob("best_model_*.pth"):
                    if old_ckpt != checkpoint_path:
                        old_ckpt.unlink()

            if early_stopping(val_acc):
                logger.info("Early stopping triggered")
                break

            # Clear cache after each epoch to prevent memory fragmentation
            if device.type == "cuda":
                torch.cuda.empty_cache()
            elif device.type == "mps":
                try:
                    torch.mps.empty_cache()
                except AttributeError:
                    pass  # MPS empty_cache not available in older PyTorch

        # INDUSTRIAL-GRADE: Training completed - generate final report
        logger.info("="*60)
        logger.info("✓ Training completed successfully")
        logger.info(f"📊 Final Results:")
        logger.info(f"  Best Val Accuracy: {best_val_acc:.2f}%")
        logger.info(f"  Total Epochs: {epoch + 1}")
        logger.info(f"  Best Checkpoint: {checkpoint_path if best_model_state else 'None'}")
        logger.info("="*60)

        # INDUSTRIAL-GRADE: Generate confusion matrix for best model
        if best_model_state:
            logger.info("Generating confusion matrix for best model...")
            model.load_state_dict(best_model_state['model_state_dict'])
            model.eval()

            all_preds = []
            all_labels = []
            with torch.no_grad():
                for images, labels in tqdm(val_loader, desc="Final Evaluation"):
                    images = images.to(device)
                    outputs = model(images)
                    _, predicted = outputs.max(1)
                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.numpy())

            # Save confusion matrix
            cm = confusion_matrix(all_labels, all_preds)
            np.save(checkpoint_dir / "confusion_matrix.npy", cm)

            # Save classification report
            # CRITICAL FIX: Only use labels that are actually present in the data
            unique_labels = sorted(list(set(all_labels + all_preds)))
            labels_subset = [i for i in range(len(TARGET_CLASSES)) if i in unique_labels]
            target_names_subset = [TARGET_CLASSES[i] for i in labels_subset]

            report = classification_report(
                all_labels, all_preds,
                labels=labels_subset,
                target_names=target_names_subset,
                output_dict=True,
                zero_division=0
            )
            with open(checkpoint_dir / "classification_report.json", "w") as f:
                json.dump(report, f, indent=2)

            logger.info(f"✓ Saved confusion matrix and classification report to {checkpoint_dir}")
            logger.info(f"  Classes present in validation: {len(unique_labels)}/{len(TARGET_CLASSES)}")

        # INDUSTRIAL-GRADE: Save final metrics
        with open(checkpoint_dir / "metrics_history.json", "w") as f:
            json.dump(metrics_history, f, indent=2)

        logger.info("✓ All artifacts saved successfully")

    except RuntimeError as e:
        if "out of memory" in str(e) or "MPS" in str(e):
            logger.error(f"OOM Error: {e}")
            logger.error("Suggestions:")
            logger.error("  1. Reduce batch_size further (try batch_size=1)")
            logger.error("  2. Reduce input_size (try 128 or 192)")
            logger.error("  3. Use a smaller model backbone (e.g., resnet50)")
            if device.type == "cuda":
                torch.cuda.empty_cache()
            elif device.type == "mps":
                try:
                    torch.mps.empty_cache()
                except AttributeError:
                    pass
        raise
    except Exception as e:
        logger.error(f"Training failed with error: {e}")
        raise
    finally:
        # Cleanup
        try:
            wandb.finish()
        except:
            pass
        if device.type == "cuda":
            torch.cuda.empty_cache()
        elif device.type == "mps":
            try:
                torch.mps.empty_cache()
            except AttributeError:
                pass

    return model


In [ ]:
# INDUSTRIAL-GRADE: Test-Time Augmentation for Inference
def predict_with_tta(model, image, device, num_augmentations=5):
    """
    Test-Time Augmentation for robust predictions.
    Applies multiple augmentations and averages predictions.

    Args:
        model: Trained model
        image: PIL Image or tensor
        device: torch device
        num_augmentations: Number of TTA iterations

    Returns:
        Averaged predictions (logits)
    """
    model.eval()

    # TTA transforms
    tta_transforms = [
        transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        for _ in range(num_augmentations)
    ]

    predictions = []
    with torch.no_grad():
        for transform in tta_transforms:
            if isinstance(image, torch.Tensor):
                img_tensor = image
            else:
                img_tensor = transform(image).unsqueeze(0).to(device)

            output = model(img_tensor)
            predictions.append(output)

    # Average predictions
    avg_prediction = torch.stack(predictions).mean(dim=0)
    return avg_prediction


In [ ]:
# INDUSTRIAL-GRADE: Model Export for Production
def export_model_for_production(model, config, checkpoint_path, export_dir="exports"):
    """
    Export model to multiple formats for production deployment.

    Exports:
    - PyTorch (.pth) - for PyTorch inference
    - TorchScript (.pt) - for C++ deployment
    - ONNX (.onnx) - for cross-platform deployment
    """
    export_dir = Path(export_dir)
    export_dir.mkdir(exist_ok=True)

    model.eval()
    device = next(model.parameters()).device

    # 1. Save PyTorch model
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
        'target_classes': TARGET_CLASSES
    }, export_dir / "model.pth")
    logger.info(f"✓ Exported PyTorch model to {export_dir / 'model.pth'}")

    # 2. Export to TorchScript
    try:
        dummy_input = torch.randn(1, 3, config['data']['input_size'], config['data']['input_size']).to(device)
        traced_model = torch.jit.trace(model, dummy_input)
        traced_model.save(export_dir / "model_torchscript.pt")
        logger.info(f"✓ Exported TorchScript model to {export_dir / 'model_torchscript.pt'}")
    except Exception as e:
        logger.warning(f"TorchScript export failed: {e}")

    # 3. Export to ONNX
    try:
        torch.onnx.export(
            model,
            dummy_input,
            export_dir / "model.onnx",
            export_params=True,
            opset_version=14,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
        )
        logger.info(f"✓ Exported ONNX model to {export_dir / 'model.onnx'}")
    except Exception as e:
        logger.warning(f"ONNX export failed: {e}")

    # 4. Save metadata
    metadata = {
        'model_name': config['model']['backbone'],
        'num_classes': config['model']['num_classes'],
        'input_size': config['data']['input_size'],
        'target_classes': TARGET_CLASSES,
        'checkpoint_path': str(checkpoint_path)
    }
    with open(export_dir / "metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)
    logger.info(f"✓ Saved metadata to {export_dir / 'metadata.json'}")

    logger.info(f"✅ Model export complete! All files in {export_dir}")


In [ ]:
# PEAK STANDARD GNN
# Using Graph Attention Networks v2 (GATv2) for superior expressive power

def generate_structured_knowledge_graph(num_classes=30, feat_dim=128):
    """
    Generates a realistic Knowledge Graph structure for waste classification.
    Simulates the schema: Item -> Material -> Bin
    """
    logger.info("Generating structured Knowledge Graph...")
    
    total_nodes = num_classes + 8 + 4
    x = torch.randn(total_nodes, feat_dim) # Node features (embeddings)
    
    edge_sources = []
    edge_targets = []
    
    # Node Indices for Materials
    mat_base = num_classes
    mat_plastic = mat_base + 0
    mat_paper = mat_base + 1
    mat_glass = mat_base + 2
    mat_metal = mat_base + 3
    mat_organic = mat_base + 4
    mat_fabric = mat_base + 5
    mat_ewaste = mat_base + 6
    mat_misc = mat_base + 7
    
    # Node Indices for Bins
    bin_base = mat_base + 8
    bin_recycle = bin_base + 0
    bin_compost = bin_base + 1
    bin_haz = bin_base + 2
    bin_landfill = bin_base + 3
    
    # 1. Edges: Material -> Bin (Knowledge Rules)
    mat_bin_map = [
        (mat_plastic, bin_recycle),
        (mat_paper, bin_recycle),
        (mat_glass, bin_recycle),
        (mat_metal, bin_recycle),
        (mat_organic, bin_compost),
        (mat_fabric, bin_landfill), 
        (mat_ewaste, bin_haz),
        (mat_misc, bin_landfill)
    ]
    
    for m, b in mat_bin_map:
        edge_sources.append(m); edge_targets.append(b)
        edge_sources.append(b); edge_targets.append(m)
        
    # 2. Edges: Item -> Material (Simulate Classification Knowledge)
    for i in range(num_classes):
        mat_idx = mat_base + (i % 8) 
        edge_sources.append(i); edge_targets.append(mat_idx)
        edge_sources.append(mat_idx); edge_targets.append(i)
        
    # 3. Edges: Item -> Item (Similarity)
    for i in range(num_classes):
        neighbor = (i + 8) % num_classes
        edge_sources.append(i); edge_targets.append(neighbor)
        edge_sources.append(neighbor); edge_targets.append(i)

    edge_index = torch.tensor([edge_sources, edge_targets], dtype=torch.long)
    
    logger.info(f"Graph generated: {total_nodes} nodes, {len(edge_sources)} edges.")
    
    return Data(x=x, edge_index=edge_index, num_nodes=total_nodes)

class GATv2Model(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=4, heads=8, dropout=0.3):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GATv2Conv(in_channels, hidden_channels, heads=heads, concat=True, dropout=dropout))
        for _ in range(num_layers - 2):
            self.convs.append(GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads, concat=True, dropout=dropout))
        self.convs.append(GATv2Conv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout))
        self.dropout = dropout
        self.norm = nn.ModuleList([nn.LayerNorm(hidden_channels * heads) for _ in range(num_layers - 1)])

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            x = self.norm[i](x)
            x = F.gelu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        return self.convs[-1](x, edge_index)

In [ ]:
def train_gnn_model():
    set_seed()
    device = get_device()
    optimize_memory(device)
    logger.info(f"Using device: {device}")

    in_dim = 128
    hidden_dim = 512
    out_dim = 256
    lr = 0.001
    epochs = 50

    data = generate_structured_knowledge_graph(num_classes=30, feat_dim=128).to(device)

    model = GATv2Model(in_dim, hidden_dim, out_dim).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

    logger.info("Starting GNN Training...")
    best_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        z = model(data.x, data.edge_index)

        pos_src, pos_dst = data.edge_index
        pos_loss = -torch.log(torch.sigmoid((z[pos_src] * z[pos_dst]).sum(dim=1)) + 1e-15).mean()

        neg_src = torch.randint(0, data.num_nodes, (pos_src.size(0),), device=device)
        neg_dst = torch.randint(0, data.num_nodes, (pos_src.size(0),), device=device)
        neg_loss = -torch.log(1 - torch.sigmoid((z[neg_src] * z[neg_dst]).sum(dim=1)) + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step(loss)

        if loss.item() < best_loss:
            best_loss = loss.item()

        if (epoch + 1) % 5 == 0:
            logger.info(f"Epoch {epoch+1}/{epochs}: Loss {loss.item():.4f}, Best Loss {best_loss:.4f}")

    return model

In [ ]:
if __name__ == "__main__":
    try:
        logger.info("="*80)
        logger.info("Phase 1: Multi-Source Data Lake Vision Training")
        logger.info("="*80)

        # CRITICAL: Starting fresh with improved hyperparameters to fix overfitting
        # Previous training: Epochs 1-6 showed severe overfitting (92.64% → 78.35%)
        # New config: Lower LR (1e-5), higher weight decay (0.1)
        # Old checkpoint available at: checkpoints/best_model_epoch1_acc92.64.pth
        vision_model = train_vision_model(VISION_CONFIG, resume_from_checkpoint=None)

        if vision_model is not None:
            save_path = "best_vision_eva02_lake.pth"
            torch.save(vision_model.state_dict(), save_path)
            logger.info(f"Vision model saved to {save_path}")

            del vision_model
            device = get_device()
            if device.type == "cuda":
                torch.cuda.empty_cache()
            elif device.type == "mps":
                try:
                    torch.mps.empty_cache()
                except AttributeError:
                    pass
        else:
            logger.error("Vision model training failed")

        logger.info("="*80)
        logger.info("Phase 2: GNN Knowledge Graph Training")
        logger.info("="*80)

        gnn_model = train_gnn_model()

        if gnn_model is not None:
            save_path = "best_gnn_gatv2.pth"
            torch.save(gnn_model.state_dict(), save_path)
            logger.info(f"GNN model saved to {save_path}")

            del gnn_model
            device = get_device()
            if device.type == "cuda":
                torch.cuda.empty_cache()
            elif device.type == "mps":
                try:
                    torch.mps.empty_cache()
                except AttributeError:
                    pass

        logger.info("="*80)
        logger.info("Training completed successfully!")
        logger.info("="*80)

    except Exception as e:
        logger.error(f"Training failed with error: {e}")
        import traceback
        traceback.print_exc()
        raise